In [ ]:
### Install necessary libraries (run in a Jupyter cell)
!pip install -q transformers datasets peft accelerate bitsandbytes torch

# Import required libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load LLaMA-3.2-3B with QLoRA
model_name = "meta-llama/Llama-3.2-3b-hf"  # Change if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load dataset (Wikitext-2 for example, replace with your dataset)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"]

# Apply LoRA configuration
lora_config = LoraConfig(
    r=8,  # Low-rank matrix size
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],  # Only tune attention layers
    bias="none"")
)

# Add LoRA adapters to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./llama3-qLoRA-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    optim="adamw_torch",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./llama3-qLoRA-finetuned")
tokenizer.save_pretrained("./llama3-qLoRA-finetuned")

# Generate text before and after fine-tuning
def generate_text(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

prompt = "Once upon a time, in the world of artificial intelligence,"
print("Generated Text:", generate_text(prompt, model, tokenizer))
